First, we import the necessary libraries.

In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv("ground_water_quality_in_rajasthan-2014.csv",
                encoding = "unicode_escape")

FileNotFoundError: [Errno 2] File b'ground_water_quality_in_rajasthan-2014.csv' does not exist: b'ground_water_quality_in_rajasthan-2014.csv'

In order to fill in missing values, we will be using linear regression. The filled features will serve as training <br>
data for the unfilled values. Here, we implement the normal equation of the linear regression model. This function <br>
will return the weights.

In [ ]:
def linRegModel(x_train, y_train):
    ones = np.ones([x_train.shape[0], 1])
    x_train = np.concatenate((ones, x_train), axis=1)
    print(x_train)
    temp_1 = np.dot(x_train.T, x_train)
   # temp_1 = temp_1.astype(np.float64)
    temp_1 = np.linalg.inv(temp_1)
    temp_2 = np.dot(x_train.T, y_train)
    theta = np.dot(temp_1, temp_2)



    return theta

The next function fits the data with the parameters.

In [ ]:
def linRegFit(x,w):
#     x = np.reshape(x,(x.shape[0],1))
    ones = np.ones([x.shape[0], 1])
    x = np.concatenate((ones, x), axis=1)
    return np.dot(x,w)
    

Code to find the columns with missing value

In [ ]:
missing_columns = df.loc[:, df.isnull().any()].columns.values
print(missing_columns)

## Deterministic Regression Imputation

To carry out deterministic regression imputation, first, t

In [ ]:
def random_imputation(df, feature):

    number_missing = df[feature].isnull().sum()
    observed_values = df.loc[df[feature].notnull(), feature]
    df.loc[df[feature].isnull(), feature + '_imp'] = np.random.choice(observed_values, number_missing, replace = True)
    
    return df

for feature in missing_columns:
    df[feature + '_imp'] = df[feature]
    df = random_imputation(df, feature)

In [3]:
df

NameError: name 'df' is not defined

In [4]:
deter_data = pd.DataFrame(columns = ["Det" + name for name in missing_columns])

NameError: name 'missing_columns' is not defined

In [563]:
for feature in missing_columns:
    
    deter_data["Det" + feature] = df[feature + "_imp"]
    parameters = list(set(df.columns) - set(missing_columns) - {feature + '_imp'})
    
    x_train = df[parameters]
    y_train = df[feature + '_imp']
    weights = linRegModel(x_train,y_train)
    
    deter_data.loc[df[feature].isnull(), "Det" + feature] = linRegFit(x_train, weights)[df[feature].isnull()]
    print(linRegFit(x_train, weights)[df[feature].isnull()])
    


[-39.]
[  8.5 -31.5]
[32.]


In [564]:
for feature in missing_columns:
        
    deter_data["Det" + feature] = df[feature + "_imp"]
    parameters = list(set(df.columns) - set(missing_columns) - {feature + '_imp'})
    
    #Create a Linear Regression model to estimate the missing data
    model = linear_model.LinearRegression()
    model.fit(X = df[parameters], y = df[feature + '_imp'])
    
    #observe that I preserve the index of the missing data from the original dataframe
    deter_data.loc[df[feature].isnull(), "Det" + feature] = model.predict(df[parameters])[df[feature].isnull()]
    print(model.predict(df[parameters])[df[feature].isnull()])

[13.]
[21. 18.]
[16.]


In [592]:
df[parameters]

,C,A_imp,B_imp,D
0,8,5.0,18.0,1
1,10,7.0,21.0,9
2,14,5.0,18.0,15
3,18,13.0,18.0,17


In [581]:
linRegFit(x_train, weights)

array([26.5, 32. , 26. , 30. ])

In [590]:
model.predict(df[parameters])

array([13.66017052, 16.        , 16.14616322, 15.        ])

In [586]:
df[feature].isnull()

0    False
1     True
2    False
3    False
Name: E, dtype: bool

In [567]:
model.coef_

array([-0.43118149, -0.0182704 ,  0.11327649,  0.36236297])

In [467]:
model.intercept_

10.70061728395061

In [462]:
df[parameters]

,C,A_imp,B_imp,D
0,8,5.0,18.0,1
1,10,7.0,21.0,9
2,14,12.0,21.0,15
3,18,12.0,18.0,17


In [463]:
x_train

,C,A_imp,B_imp,D
0,8,5.0,18.0,1
1,10,7.0,21.0,9
2,14,12.0,21.0,15
3,18,12.0,18.0,17


In [464]:
y_train

0    14.0
1    16.0
2    16.0
3    15.0
Name: E_imp, dtype: float64